## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import numpy as np
import gmaps.datasets
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,39.79,79,92,18.50,light rain
1,1,Nanortalik,GL,60.1432,-45.2371,41.22,80,51,12.08,broken clouds
2,2,Naftalan,AZ,40.5067,46.8250,85.42,37,32,10.16,scattered clouds
3,3,Chuy,UY,-33.6971,-53.4616,50.00,59,5,15.84,clear sky
4,4,Lebu,CL,-37.6167,-73.6500,48.45,62,0,11.61,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Naftalan,AZ,40.5067,46.8250,85.42,37,32,10.16,scattered clouds
5,5,Saint George,US,37.1041,-113.5841,82.80,22,0,3.44,clear sky
6,6,Nouadhibou,MR,20.9310,-17.0347,75.18,69,0,18.41,clear sky
8,8,Disna,BY,55.5680,28.1971,79.57,54,19,2.64,few clouds
10,10,Padang,ID,-0.9492,100.3543,77.05,87,100,3.51,overcast clouds
29,29,Itarema,BR,-2.9248,-39.9167,83.53,69,62,13.87,broken clouds
30,30,Brigantine,US,39.4101,-74.3646,75.20,56,0,11.50,clear sky
31,31,Tanete,ID,-3.9443,119.7848,75.07,96,96,3.36,overcast clouds
32,32,Kimbe,PG,-5.5502,150.1429,77.29,87,54,0.45,light rain
33,33,Ponta Do Sol,PT,32.6667,-17.1000,83.71,63,0,1.34,clear sky


In [5]:
preferred_cities_df.count()

City_ID                254
City                   254
Country                253
Lat                    254
Lng                    254
Max Temp               254
Humidity               254
Cloudiness             254
Wind Speed             254
Current Description    254
dtype: int64

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                253
City                   253
Country                253
Lat                    253
Lng                    253
Max Temp               253
Humidity               253
Cloudiness             253
Wind Speed             253
Current Description    253
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Naftalan,AZ,85.42,scattered clouds,40.5067,46.8250,
5,Saint George,US,82.80,clear sky,37.1041,-113.5841,
6,Nouadhibou,MR,75.18,clear sky,20.9310,-17.0347,
8,Disna,BY,79.57,few clouds,55.5680,28.1971,
10,Padang,ID,77.05,overcast clouds,-0.9492,100.3543,
29,Itarema,BR,83.53,broken clouds,-2.9248,-39.9167,
30,Brigantine,US,75.20,clear sky,39.4101,-74.3646,
31,Tanete,ID,75.07,overcast clouds,-3.9443,119.7848,
32,Kimbe,PG,77.29,light rain,-5.5502,150.1429,
33,Ponta Do Sol,PT,83.71,clear sky,32.6667,-17.1000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        # Grab the first hotel from the results and store the name.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
            print("Hotel not found...skipping")
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [10]:
hotel_df.dtypes

City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [11]:
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Naftalan,AZ,85.42,scattered clouds,40.5067,46.8250,Gashalti Health Hotel - Naftalan
5,Saint George,US,82.80,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
6,Nouadhibou,MR,75.18,clear sky,20.9310,-17.0347,El Medina
8,Disna,BY,79.57,few clouds,55.5680,28.1971,
10,Padang,ID,77.05,overcast clouds,-0.9492,100.3543,Mercure Padang
29,Itarema,BR,83.53,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
30,Brigantine,US,75.20,clear sky,39.4101,-74.3646,Legacy Vacation Resorts - Brigantine Beach
31,Tanete,ID,75.07,overcast clouds,-3.9443,119.7848,Wisma Tri Multi
32,Kimbe,PG,77.29,light rain,-5.5502,150.1429,Kimbe Bay Hotel
33,Ponta Do Sol,PT,83.71,clear sky,32.6667,-17.1000,Hotel do Campo


In [12]:
# 7. Drop the rows where there is no Hotel Name.

#replace empty strings as Nan
hotel_df['Hotel Name'].replace('',np.nan, inplace=True)
hotel_df_clean = hotel_df.dropna()
hotel_df_clean

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Naftalan,AZ,85.42,scattered clouds,40.5067,46.8250,Gashalti Health Hotel - Naftalan
5,Saint George,US,82.80,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
6,Nouadhibou,MR,75.18,clear sky,20.9310,-17.0347,El Medina
10,Padang,ID,77.05,overcast clouds,-0.9492,100.3543,Mercure Padang
29,Itarema,BR,83.53,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
...,...,...,...,...,...,...,...
670,Bastia,FR,83.53,clear sky,42.6667,9.3333,Hotel Santa Maria
676,Mindelo,CV,80.15,few clouds,16.8901,-24.9804,Hotel Gaudi
684,Lufilufi,WS,81.21,few clouds,-13.8745,-171.5986,Leuaina Beach Resort and Spa
689,Corinto,GR,85.89,clear sky,37.9407,22.9573,Ephira Hotel


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df_clean.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df_clean[["Lat", "Lng"]]

In [15]:
max_temp = hotel_df_clean["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info )
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))